<a href="https://colab.research.google.com/github/jadenfang/NextWordPredictor/blob/main/Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [62]:
import random

text = "Once upon a time"
model_text = text
output_text = text

newline_token_id = tokenizer.encode("\n")[0]
temperature = 0.8
k = 10
num_lines = 4
initial_words = 4

print("\nGenerating poem...\n")

for line in range(num_lines):
    target_words = random.randint(5, 7)

    # Initial line length
    if line == 0:
        words_on_line = initial_words
    else:
        words_on_line = 0

    print(f"--- Line {line + 1} (target {target_words} words) ---")

    # Generate words until 5-7 words
    while words_on_line < target_words:
        input_ids = tokenizer.encode(model_text, return_tensors="pt")

        with torch.no_grad():
            outputs = model(input_ids)

        next_token_logits = outputs.logits[0, -1, :]
        next_token_logits /= temperature

        # HARD BAN newline
        next_token_logits[newline_token_id] = -1e9

        next_token_probs = torch.softmax(next_token_logits, dim=0)

        # Check top K tokens
        top_probs, top_indices = torch.topk(next_token_probs, k)
        top_probs = top_probs / top_probs.sum()

        next_token_id = top_indices[torch.multinomial(top_probs, 1)].item()
        next_word = tokenizer.decode([next_token_id])

        # Skip pure whitespace or punctuation-only tokens
        if not next_word.strip() or not any(c.isalnum() for c in next_word):
            continue

        model_text += next_word
        output_text += next_word

        words_on_line += 1

    output_text += "\n"

print("\nFinal poem:\n")
print(output_text)



Generating poem...

--- Line 1 (target 6 words) ---
--- Line 2 (target 6 words) ---
--- Line 3 (target 5 words) ---
--- Line 4 (target 7 words) ---

Final poem:

Once upon a time I thought
 I would be in love and
 had the opportunity to meet
 and talk to a couple of friends

